In [23]:
import re
import traceback
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from sklearn.metrics import accuracy_score
import math
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import sqlite3
from collections import Counter
import ast
from ast import literal_eval
from Content_based import ContentRecommender
from Filter_based import FilterRecommender 

In [24]:
class HybridRecommender:
    def __init__(self, database):
        self.ct = ContentRecommender(database)
        self.fr = FilterRecommender(database)

    def recommend_hybrid(self, city, roomtype, amenities, min_price, max_price, days):
        filtered_hotels = self.get_filterd_data(city, roomtype, amenities, min_price, max_price, days)
        
        if filtered_hotels.empty:
            print('No hotels available after filtering')
            return pd.DataFrame()

        listing_ids = filtered_hotels['listing_id'].tolist()
        print(listing_ids)
        similar_hotels = self.ct.recommend_by_amenities_topic(listing_ids)

        return similar_hotels
    
    def get_filterd_data(self, city, roomtype, amenities, min_price, max_price, days):
        filtered_hotels = self.fr.filter_by_all(city, roomtype, amenities, min_price, max_price, days)
        return filtered_hotels

In [25]:
# Khởi tạo cơ sở dữ liệu
database = 'airbnb_data.db'

# Khởi tạo HybridRecommender
hr = HybridRecommender(database)

# Các thông số đầu vào để lọc và khuyến nghị
city = "Washington DC"
roomtype = "Private room"
amenities = "Wifi"
min_price = 50
max_price = 200
days = 3

# Lấy khuyến nghị
recommendations = hr.recommend_hybrid(city, roomtype, amenities, min_price, max_price, days)
print(recommendations)


[426332090, 360451390, 401839580, 410628830, 36746760, 132477880, 386878420, 138650750, 158209830, 371795490, 73879560, 114553210, 122020790, 343031240, 203027450, 129736070, 525792000, 247558700, 411515960, 294872410, 157904490, 168117700, 406565490, 399636050, 9454260, 224166820, 129265450, 235077840, 256058610, 108609970, 187460310, 308241110, 88465420, 182374920, 159628710, 60913880, 352308240, 201263410, 118094150, 453136860, 398365310, 164654850, 159637880, 374936320, 167534380, 27910500, 60840640, 163813170, 163805620, 83530340, 372896430, 115136800, 374552360, 407485250, 179566980, 153285330, 405355740, 405619000, 8088300, 406204540, 195996460, 441199040, 441196830, 47301240, 188682000, 8284970, 348682380, 475686630, 249451140, 117143380, 411525120, 67759300, 454220870, 540166280, 3134050, 69566850, 182655370, 313908650, 507727510, 31032760, 411528870, 416462130, 64042020, 22854310, 2990300, 520563200, 48487990, 44829450, 23657260, 496600330, 1640690, 38517030, 310889800, 44120

TypeError: ContentRecommender.recommend_by_amenities_topic() takes 1 positional argument but 2 were given

In [26]:
minprice = 30
maxprice = 50
city = "washington dc"
roomtype = "private room"
amenities = "Wifi"
days = 14

result8=hr.fr.filter_by_all(city,roomtype,amenities,minprice,maxprice,days)
result8.head(5)

,listing_id,city,name,review_scores_rating,room_type,amenities,minimum_nights,listing_url,price
2176,536992420,washington dc,One-Room Getaway within Shared DC Urban Oasis,5,private room,"[""Microwave"", ""Dining table"", ""Free street par...",5,https://www.airbnb.com/rooms/53699242,44.0
1214,334327780,washington dc,"very quiet, clean and safe.",5,private room,"[""Hair dryer"", ""Essentials"", ""First aid kit"", ...",7,https://www.airbnb.com/rooms/33432778,50.0
1948,503333340,washington dc,Cozy room washing Dc near metro (9),4.94,private room,"[""Microwave"", ""Dining table"", ""Hair dryer"", ""F...",1,https://www.airbnb.com/rooms/50333334,44.0
393,116897500,washington dc,"Clean,Cozy, and Affordable-1",4.93,private room,"[""Microwave"", ""Dining table"", ""Free street par...",14,https://www.airbnb.com/rooms/11689750,49.0
764,201255500,washington dc,Private Room #2 15 mins to White House,4.92,private room,"[""Hair dryer"", ""Free street parking"", ""Private...",1,https://www.airbnb.com/rooms/20125550,31.0
